In [1]:
import os, sys
import numpy as np

import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt

from dask.distributed import Client, LocalCluster
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=16)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.156.19:8787/status,
Dashboard: http://137.129.156.19:8787/status,Workers: 1
Total threads: 16,Total memory: 94.31 GiB
Status: running,Using processes: False
Comm: inproc://137.129.156.19/54892/1,Workers: 1
Dashboard: http://137.129.156.19:8787/status,Total threads: 16
Started: Just now,Total memory: 94.31 GiB
Comm: inproc://137.129.156.19/54892/3,Total threads: 16
Dashboard: http://137.129.156.19:45917/status,Memory: 94.31 GiB
Nanny: None,


In [2]:
year = np.arange(1990,2021)
exp = 'era5_sfc_tcwv_0.25_'
indir = '/cnrm/tropics/commun/DATACOMMUN/ERA5/0.25/netcdf/sfc_1h/'
infile = indir+exp+'.nc'


In [3]:
datasets = []
datasetsL = []

def isLeapYear (yearN):
    if ((yearN % 4 == 0) and (yearN % 100 != 0)) or (yearN % 400 == 0):
        reponse = True
    else:
        reponse = False
    print(reponse, '\n')
    return reponse
        
        
for f in year:
    ds = xr.open_dataset(indir+exp+str(f)+'.nc', chunks={'time': 1},
                         )
    print(ds.time.dt.year.values[0])
    if isLeapYear(ds.time.dt.year.values[0]) == False :
        datasets.append(ds)
    else :
        datasetsL.append(ds)
ds = xr.concat(datasets, dim='time', coords='minimal', compat='override')
dsL = xr.concat(datasetsL, dim='time', coords='minimal', compat='override')

#Climatology mean of the day
ds = ds.groupby('time.dayofyear').mean('time')
dsL = dsL.groupby('time.dayofyear').mean('time')

1990
False 

1991
False 

1992
True 

1993
False 

1994
False 

1995
False 

1996
True 

1997
False 

1998
False 

1999
False 

2000
True 

2001
False 

2002
False 

2003
False 

2004
True 

2005
False 

2006
False 

2007
False 

2008
True 

2009
False 

2010
False 

2011
False 

2012
True 

2013
False 

2014
False 

2015
False 

2016
True 

2017
False 

2018
False 

2019
False 

2020
True 



In [4]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, dayofyear: 365)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
Data variables:
    tcwv       (dayofyear, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>

In [5]:
from scipy.fftpack import fftfreq, fft, fft2, fftshift
from scipy.signal import tukey, detrend
from scipy.interpolate import interp2d

def compute_2d_spectrum(F,nx,ny,dx,dy):
    """
    Compute 2d spectrum by using the 
    2d fast fourier transform
    """
    ### detrend
    Fdtr = detrend(detrend(F,axis=1),axis=0)

    ## Window Tapering
    cff_tukey = 0.5
    wdw_y = tukey(ny, cff_tukey)
    wdw_x = tukey(nx, cff_tukey)
    wdw = np.outer(wdw_x,wdw_y)
    Ftpr = Fdtr * wdw[:,:]

    ### spectre 2d
    Fsp = np.zeros(Ftpr.shape)
    ld = 1./dy * 1./dx * (wdw**2).sum()

    pu = fftshift(fft2(Ftpr))
    Fsp = (np.conjugate(pu)*pu).real/ ld

    ### Wavelength and period
    kx = fftshift(fftfreq(nx,dx))
    ky = fftshift(fftfreq(ny,dy))

    return kx, ky, Fsp

In [ ]:

### Resolution
dx = 0.25*111000
dt = ds['dayofyear'].diff("dayofyear").mean().values.astype('timedelta64[s]').astype(np.float32)

fx = 1./dx
ft = 1./dt

tcwv = ds.tcwv.values

for y_ind in range(tcwv.shape[1]):
    nt,nx = u_rgrd[:,y_ind,:].shape

    w,kx, Spu = compute_2d_spectrum(tcwv[:,y_ind,:],nt,nx,dt,dx)
    
    Fsp.append(Spu)

Fsp = np.mean(np.array(Fsp),0)

In [ ]:
Fsp